## MobileNet v2
- Paper : MobileNets: (2019)_MobileNet V2-Inverted Residuals and Linear Bottlenecks

- 최근 네트워크 및 작업들은 높은 연산량과 리소스를 필요로 함
- Nasnet 계열 or 다른 모델들도 있지만 이는 모델의 복잡성 때문에 적합하지 않다
- **기존 Mobilenet v1의 구조를 따르며Linear Bottlenecks과 Inverted Residuals 구조 도입을 통해 mobilenet v1의 성능을 개선**
- 신규 구조에 대한 해석을 실험적으로 검증함
- **MobileNet v2를 backbone으로 이용한 네트워크들의 object detection과 segmentation에서의 성능이 이 시점 (CVPR 2018)의 다른 backbone 네트워크 보다 더 낫다고 설명**

## Recap MobileNet v1
- Depthwise Separable Convolution 
    - Depthwise conv하고 concat 한다음에 Pointwise convolution 을 진행함
    - 3x3 과 1x1 사이에 bn과 relu도 넣고, 단 xception은 사이에 relu를 넣지 않음
    - 결과는 3x3 conv 한 것과 같지만 연산량이 더 줄어든다

![depthwise_separable_conv](img/depthwise_separable_conv.png)

<hr>

### MobileNet v2 요약
- MobileNet v1에서 Pointwise에 계산량 부담 인식하고, 1x1 pointwise conv -> depthwise conv -> projection layer를 통해 다시 input channel의 갯수로 줄이는 bottleneck의 구조를 가짐

- Linear Bottleneck
   - 뉴럴 네트워크의 manifold(고차원 데이터의 특정 정보들이 저차원 영역으로 매핑되는 현상) 존재
   - relu는 양수의 값을 단순히 그대로 전파하므로(linear transformation임) manifold정보를 그대로 유지한다고 볼 수 있음
   - linear bottleneck layer란 걸 만들어서 차원을 줄이고 manifold의 중요한 정보들을 그대로 유지

- Inverted Residuals
   - Residual module (stride1, relu6) 3x3 -> 1x1 -> residual add 
   - 일반 module (stride2, relu6) 3x3 -> 1x1  총 2종류 module 사용
   - narrow에 해당하는 저차원의 layer에는 필요한 정보만 압축되어서 저장되어 있으며 필요한 정보는 narrow에 있기 때문에, skip connection으로 사용해도 필요한 정보를 더 깊은 layer까지 잘 전달할 것임
   - 기존 wide -> narrow(좁게) -> wide 형태를 narrow -> wide -> narrow 로 바꾸며 wide를 skip connection 하던걸 / narrow를 skip connection 함
   
<hr>

## MobileNet v1 vs v2

![mobilenet_v1_vs_mobilenet_v2](img/mobilenet_v1_vs_mobilenet_v2.png)

- MobileNet v1
    - v1 block은 1가지 종류, 2개의 layer로 구성
    - 첫번째 layer: depthwise convolution
        - 각 인풋 채널에 single convolution filter를 적용하여 네트워크 경량화
    - 두번째 layer: pointwise convolution
        - depthwise convolution의 결과(concat된걸) 를 pointwise convolution(1x1) 한 것
    - Activation function :  Relu6를 사용한 block을 구성
        - relu6 : min(max(x, 0), 6) / 음수는 0, 6이상의 값은 6으로 수렴

- MobileNet v2
    - v2 block은 2가지 종류의 block 존재 및 각각의 block은 3개의 layer를 가짐
        - 첫번째 block
            - stride가 1인 residual block
            - 첫 번째 layer는 pointwise(1x1)
            - 두번째 layer는 depthwise convolution - stride:1
            - (linear)세번째 layer에서는 다시 pointwise(1x1) convolution / not activation function
            - skip connection 존재 (o)
            
        - 두번째 block
            - downsizing 목적의 stride가 2인 block
            - 첫 번째 layer는 pointwise(1x1)
            - 두번째 layer는 depthwise convolution - stride:2 (다운sizing)
            - (linear)세번째 layer에서는 다시 pointwise(1x1) convolution / not activation function
            - skip connection 존재하지 않음 (x) -> stride 2를 통해 feature가 줄어드니 skipconnection 또한 크기 맞춰줘야되서 그냥 적용하지 않은 것으로 추정
            
            
<hr>


## MobileNet v2 Architecture

![mobilnet_v2_block_architecture](img/mobilnet_v2_block_architecture.png)
- T : 입출력의 크기에 상수배, expansion factor
- 논문에서는 6으로 실험 -> 입력의 채널이 64라면 출력의 채널은 384 (일반적으로 5~10사이 권장)

![mobilenet_v2_architecture](img/mobilenet_v2_architecture.png)

- t:expansion factor / c:output channel의 수 / n:반복 횟수 / s:stride
- 입력 resolution을 기준으로 multiply-add에 대한 연산 비용은 약 300M / 파라미터의 갯수는 3.4M

- 다양한 실험 parameter
    - Width multiplier는 layer의 채널수를 일정 비율로 줄이는 역할 
    - Resolution multiplier는 입력 이미지의 resolution을 일정한 비율로 줄이는 것


<hr>


## Input size에 따른 메모리 비교
![mobilenet_v1_v2_shufflenet_memory](img/mobilenet_v1_v2_shufflenet_memory.png)

- Mobilenet v2의 크기가 가장 작음

<hr>

## Manifold

![manifold](img/manifold.png)
- 고차원의 데이터가 저차원으로 압축되면서 특정 정보들이 저차원의 어떤 영역으로 매핑이 되는 현상
- 뉴럴 네트워크에서 일반적으로 고차원 -> 저차원으로 압축하는 Encoder 역할의 네트워크 부분이 있음 (Feature Extractin 과정) -> 따라서 뉴럴 네트워크의 manifold는 저차원의 subspace로 매핑이 가능하다
    - 이는 데이터의 manifold가 Relu 를 통과하고 나서도 입력값이 음수가 아니라서 0이 되지 않은 상태라면, ReLU는 linear transformation 연산을 거친 것이라고 말할 수 있음
    - 즉 Relu를 거치는 것은 identity matrix를 곱한것과 같아 단순 linear transformation 이라고 할수 있음

- **최종 정리 : 뉴럴 네트워크는 네트워크를 거치며, 저차원으로 매핑이 되는 연산이 계속 되는데, 이 때, (인풋의 manifold가 인풋 space의 저차원 subspace에 있다는 가정 하) ReLU는 양수의 값을 단순히 그대로 전파하므로 (즉, linear transformation이므로), manifold 상의 정보를 그대로 유지 한다고 볼 수 있음**

<hr>


## MobileNet v2 주요아이디어 1) Linear bottleneck
- 이전 MobileNetV1의 구조였던 Depthwise Separable Convolution에서 Pointwise Conv의 연산량이 많아지기 때문에 계산량 부담을 인식하고 Depthwise Convolution에 연산 비중을 올리는 테크닉 제안

- 따라서 1x1 pointwise conv -> depthwise conv -> projection layer를 통해 다시 input channel의 갯수로 줄이는 bottleneck의 구조를 가짐

![mobilenet_v2_linear_bottleneck_block](img/mobilenet_v2_linear_bottleneck_block.png)

- 저차원으로 매핑하는 bottleneck architecture를 만들 때, linear transformation 역할을 하는 linear bottleneck layer를 만들어서 차원을 줄이고 manifold 상의 중요한 정보들은 그대로 유지해보자는 것의 컨셉

### Relu6
- ReLU6는 positive region에서 Maximum value가 6을 못넘어가는 activation function
- 기존의 ReLU는 Positive region에서 상한선이 없기 때문에 이를 표현하기 위해서 bit수를 정할 때 risk가 존재
- 하지만 ReLU6는 결과값이 6이하기 때문에 integer의 bit를 3bit로 고정하여 연산량 감소의 효과 존재

![mobilenetv2_linear_bottleneck](img/mobilenetv2_linear_bottleneck.png)

<hr>

## MobileNet v2 주요아이디어 2) Inverted Residuals

### 1. Basic Residuals
![mobilenetv2_residuals](img/mobilenetv2_residuals.png)

- 일반적인 Residual block 형태
- 처음에는 채널이 많은 구조에서 1x1으로 줄이고 뒤에서 Expansion을 하는 것이 일반적
- wide -> narrow -> wide 형태가 됨
- 처음에 들어오는 입력은 채널이 많은 wide한 형태 -> 1x1 convolution을 이용하여 채널을 줄임 ->  다음 layer에서 bottleneck을 만듬 -> bottleneck에서는 3x3 convolution을 이용한 convolution 연산을 하게 됨 ->  다시 skip connection과 합쳐지기 위한 원래의 사이즈 복원

### 2. Inverted Residuals
![mobilenetv2_inverted_residuals1](img/mobilenetv2_inverted_residuals1.png)

- inverted residual은 residual block과 정반대로 움직임
- 처음 입력으로 그려진 점선 형태의 feature는 앞에서 다룬 linear bottleneck (no relu)
-  narrow → wide → narrow 구조로 skip connection을 합침

- **선택 이유**
    - **narrow에 해당하는 저차원의 layer에는 필요한 정보만 압축되어서 저장되어 있다**
    - **따라서 필요한 정보는 narrow에 있기 때문에, skip connection으로 사용해도 필요한 정보를 더 깊은 layer까지 잘 전달할 것**
    - **주요 목적은 압축된 narrow layer를 skip connection으로 사용하여  메모리 사용량을 줄이기 위함**

<hr>


## 연산량 (Operations)
- 인풋의 크기 (h x w) / expansion factor (t) / kernel-size (k) / input channel 수 (d') / 아웃풋 채널수 (d'')

![mobilnet_v2_operations](img/mobilnet_v2_operations.png)

- 파란색 : 아웃풋에 관련된 식
- 빨간색 : 아웃풋을 계산하기 위한 convolution filter의 사이즈

- 첫번째 식은 1x1 convolution을 곱하기 때문에 1x1xd' 가 아웃풋에 곱해짐
- 두번째 식은 3x3 convolution을 depthwise convolution 하는 연산 -> 곱해지는 채널의 수가 1으로 생략
- 세번째 식은 마지막 출력이므로 아웃풋 채널인 d'' 도입

<hr>


## 결과 (Result)

![mobilenet_v2_result](img/mobilenet_v2_result.png)
- Params와 MAdds, 연산속도를 획기적으로 줄이면서, 기존의 MobileNet, ShuffleNet보다 더 좋은 성능을 보여줌
- Detection, segmentation 에서도 적은 파라미터로 우수한 성능을 보임

<hr>



## 참고문헌
- https://gaussian37.github.io/dl-concept-mobilenet_v2/
- https://www.youtube.com/watch?v=mT5Y-Zumbbw&t=1397s